In [1]:
import numpy as np
import pandas as pd
from tpot import TPOTClassifier

from dask.distributed import Client
import joblib

In [2]:
df_temp = pd.read_csv("data/base.csv", sep=';')
df = df_temp[['VAR01', 'VAR02', 'VAR03', 'VAR04', 'VAR05',
              'VAR06', 'VAR07', 'VAR08', 'VAR09', 'VAR10',
              'VAR12', 'VAR14', 'VAR15', 'VAR16', 'VAR17',
              'VAR18', 'VAR19', 'VAR20', 'VAR21', 'VAR22',
              'VAR23', 'VAR24', 'VAR25', 'VAR28', 'VAR30',
              'VAR31', 'VAR31b', 'VAR31c', 'VAR32a', 'VAR32b',
              'VAR32c', 'VAR32d', 'VAR33', 'VAR34', 'VAR35', 'DESEMPENHO_BINARIO', 'ID do Aluno']]

df = df.rename(columns={"DESEMPENHO_BINARIO": "Y"})

In [3]:
df["VAR17"] = df["VAR17"].astype(str).replace(',','.')
df["VAR24"] = df["VAR24"].astype(str).replace(',','.')
df["VAR25"] = df["VAR25"].astype(str).replace(',','.')
df["VAR30"] = df["VAR30"].astype(str).replace(',','.')

In [4]:
df["VAR17"] = df["VAR17"].astype(float)
df["VAR24"] = df["VAR24"].astype(float)
df["VAR25"] = df["VAR25"].astype(float)
df["VAR30"] = df["VAR30"].astype(float)

In [5]:
msk = np.random.rand(len(df)) < 0.7

In [6]:
train = df[msk]
test = df[~msk]
del test['Y']

In [7]:
IDtest = test["ID do Aluno"].reset_index()
test = test.drop(labels = ["ID do Aluno"],axis = 1)

In [8]:
Y_train = train["Y"]
X_train = train.drop(labels = ["Y"],axis = 1)

In [9]:
pipeline_optimizer = TPOTClassifier()

In [10]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)

In [11]:
pipeline_optimizer.fit(X_train, Y_train)

Generation 1 - Current best internal CV score: 0.8871493469987939
Generation 2 - Current best internal CV score: 0.8871493469987939
Generation 3 - Current best internal CV score: 0.8871493469987939
Generation 4 - Current best internal CV score: 0.8871493469987939
Generation 5 - Current best internal CV score: 0.8880479828762657

Best pipeline: ExtraTreesClassifier(CombineDFs(input_matrix, SGDClassifier(VarianceThreshold(input_matrix, threshold=0.001), alpha=0.01, eta0=0.1, fit_intercept=True, l1_ratio=0.75, learning_rate=constant, loss=perceptron, penalty=elasticnet, power_t=0.1)), bootstrap=False, criterion=gini, max_features=0.6500000000000001, min_samples_leaf=14, min_samples_split=6, n_estimators=100)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=20,
               random_state=42, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [12]:
# connect to the cluster
client = Client(processes=False) 

# create the estimator normally
estimator = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=3, 
                                    use_dask=True,
                                    periodic_checkpoint_folder="data/tpot-output/")

# perform the fit in this context manager
with joblib.parallel_backend("dask"):
    estimator.fit(X_train, Y_train)

31 operators have been imported by TPOT.


Saving periodic pipeline from pareto front to data/tpot-output/pipeline_gen_1_idx_0_2020.03.25_00-26-02.py
_pre_test decorator: _random_mutation_operator: num_test=0 [00:26:02] /private/var/folders/kg/56jlft1d2ts2r4l5v96sv18h0000gn/T/pip-install-_qm87u0a/xgboost/xgboost/src/learner.cc:946: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000013280c9e0 dmlc::LogMessageFatal::~LogMessageFatal() + 112
  [bt] (1) 2   libxgboost.dylib                    0x00000001328a7576 xgboost::LearnerImpl::ConfigureNumFeatures() + 822
  [bt] (2) 3   libxgboost.dylib                    0x000000013289decf xgboost::LearnerImpl::Configure() + 1119
  [bt] (3) 4   libxgboost.dylib                    0x000000013289e459 xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 121
  [bt] (4) 5   libxgboost.dylib                    0x0000000132805e8a XGBoosterUpdateOneIter + 122

distributed.client - ERROR - Error in callback <function DaskDistributedBackend.apply_async.<locals>.callback_wrapper at 0x144d53440> of <Future: cancelled, key: _tree_query_parallel_helper-batch-75eff486e95a454896eecf58bca18a3b>:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/distributed/client.py", line 285, in execute_callback
    fn(fut)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/joblib/_dask.py", line 260, in callback_wrapper
    result = future.result()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/distributed/client.py", line 222, in result
    raise result
concurrent.futures._base.CancelledError: _tree_query_parallel_helper-batch-75eff486e95a454896eecf58bca18a3b
distributed.client - ERROR - Error in callback <function DaskDistributedBackend.apply_async.<locals>.callback_wrapper at 0x142253830> of <Future: cancelled, key: _t

_pre_test decorator: _random_mutation_operator: num_test=0 _tree_query_parallel_helper-batch-75eff486e95a454896eecf58bca18a3b.
Generation 5 - Current Pareto front scores:
-1	0.8885965845924328	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=10, XGBClassifier__min_child_weight=6, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.6000000000000001)
-2	0.8886435219310858	XGBClassifier(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures__include_bias=False, PolynomialFeatures__interaction_only=False), XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=10, XGBClassifier__min_child_weight=6, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.6000000000000001)
-3	0.893619882480363	XGBClassifier(StandardScaler(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures__include_bias=False, PolynomialFeatures__interaction_only

In [14]:
print(estimator)

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder='data/tpot-output/',
               population_size=20, random_state=42, scoring=None, subsample=1.0,
               template=None, use_dask=True, verbosity=3, warm_start=False)
